# Webscrapping with Beautiful Soup.

This example shows how to download any ZIP file listed on a given webpage.

In [1]:

import requests
#import urllib
import os
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
# Create destination
cwd = os.getcwd()
datadir = cwd +"/data"
print("The current Working directory is " + cwd)
if not os.path.exists(datadir):
    os.mkdir(datadir) #, 0777);
#print "Created new directory " + newdir
#newfile = open('zipfiles.txt','w')

The current Working directory is C:\source\HACK\chart-digitizer


In [3]:
def scrape_find_zips(url):
    """ scrape the given url for any zip files in href tags """
    response = requests.get(url)
    #print(response.status_code)
    #print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')
    all_tags = soup.find_all('a', href=True)
    tag_dict = {}
    for tag in all_tags:
        if '.tar.gz' in tag.get_text() or '.zip' in tag.get_text() or 'py2.py3-none-any.whl' in tag.get_text():
            tag_dict[tag.get_text()] = tag['href']
    return tag_dict


In [4]:
base =  "http://www.nemweb.com.au"
url = "http://www.nemweb.com.au/Reports/Current/Daily_Reports/" #Did previously
url = "http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/"

tag_dict = scrape_find_zips(url)

print(next (iter (tag_dict.values())))
print(next (iter (tag_dict.keys())))

dest=os.getcwd()+'/data/'

/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170301.zip
PUBLIC_DAILY_20170301.zip


In [5]:
def scrape_downloader(url, urls, dest='./data/'):
    """ download all files listed in urls {} dict 
    Input:
        url = base url
        urls= target paths to append to base url
        dest= local path to store files
    """
    for k_name, v_path in urls.items():
        src = url+v_path
        tgt = dest+k_name
    
        r = requests.get(src)
        with open(tgt, 'wb') as code:
            code.write(r.content)
        print(src)
    

In [6]:
scrape_downloader(base, tag_dict, dest='./data/')

http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170301.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170401.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170501.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170601.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170701.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170801.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20170901.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20171001.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20171101.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20171201.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20180101.zip
http://www.nemweb.com.au/Reports/ARCHIVE/Daily_Reports/PUBLIC_DAILY_20180201.zip
http://www.nemweb.com.au/Rep

#Setup
Setup folder to  unzip  into


In [ ]:
# Remember that datadir = cwd +"/data"
unzip_dir=os.getcwd()+'/data/unzip/'
if not os.path.exists(unzip_dir):
    os.mkdir(unzip_dir)
    print('Created',unzip_dir)

In [ ]:
 
f_zip = [datadir + '/' + x for x in os.listdir('./data/')]

In [ ]:
print(f_zip[0])

In [ ]:
import zipfile
for f in f_zip[0:3]:  #remove [0] to extend loop
    print(f)
    if f.endswith(".zip"):
        #Unzip
        zip_ref = zipfile.ZipFile(f, 'r')
        zip_ref.extractall(unzip_dir)
        zip_ref.close()



In [ ]:
print(root)
print(dirs)
print(files)

In [ ]:

f_unzip = os.listdir('./data/unzip/')

In [ ]:
print(unzip_dir+f_unzip[0])

In [ ]:
print(f_unzip)

# Process CSV file looking for specific header

In [ ]:

import csv
#----------------------------------------------------------------------
def csv_reader(file_obj, segment=["I","TUNIT","","1"]):
    """
    Read a csv file
    scan for segment beginning with I in row n colunm 1 and TUNIT or DREGION etc in column3 
    e.g. segment=["I","TUNIT",None,"1"] for power generator data
    e.g. segment=["I","DREGION",None,"2"] for state totals
    """
    #reader = csv.DictReader(file_obj, delimiter=',', dialect=csv.unix_dialect)
    reader = csv.reader(file_obj, delimiter=',', dialect=csv.unix_dialect)
    lines=[]
    #save contents? only when found valid header
    valid=False
    for row, line in enumerate(reader):
        #print(row, line[0:9])          #Uncomment to debug.
        #Find "I" NemWeb header rows for 'T-Units'
        if line[0]==segment[0] and line[1]==segment[1] and line[3]==segment[3]:
            print("Found Start at:",row, line[0:9])
            valid=True
            start=row
            headers = line
        if line[0]==segment[0] and valid and row>start: # and line[1]=="TUNIT" and line[3]=="2":
            print("Found next header at",row,"exiting with",row-start,"lines")
            break
            
        if valid and not line[0]=='I':
            # save non header row
            lines.append(line)
    return lines, headers

In [ ]:
# create list of file paths
csv_paths = [unzip_dir+f for f in f_unzip[0:]]  # 0:3 for 3 files
print(csv_paths)

In [ ]:
#Create empty lists
newlines = lines = header = all_lines = []

for csv_path in csv_paths:
    print("Processing",csv_path)
    with open(csv_path, "r", newline='\n') as f_obj:
        # send file object to reader and extract valid rows of data
        newlines, header = csv_reader(f_obj,segment=["I","DREGION",None,"2"])
        lines.extend(newlines)
        #note lines.append creates a list of lists, rather than just one long list.

 


In [ ]:
# Now we could create new csv, store data in hadoop, or in a dataframe
print(header)
print(lines[0])

df = pd.DataFrame(data=lines, columns=header)

# Define our index as date and time column
df.set_index("SETTLEMENTDATE", inplace=True)
df.index = pd.to_datetime(df.index)

In [ ]:
# chart if you must?
#df.pivot(columns='DUID', values='TOTALCLEARED').astype(float).plot(title="Energy",legend=False)

In [ ]:

from IPython.display import display
display(df.iloc[0:11,[0,5,6,7,11,12]])


In [ ]:
#df.dtypes

In [ ]:
#Save to Hadoop File for fast loading later
df.to_hdf('AllRows.hdf5',key='NRG')

In [ ]:
nsw = df.loc[df.loc[:,"REGIONID"]=="NSW1",:]

In [ ]:
display(nsw["TOTALDEMAND"].astype(float))

In [ ]:

daily = nsw["TOTALDEMAND"].astype(float).fillna(0).resample(rule="D").sum().dropna()
    

In [ ]:
daily.head(20)

In [ ]:
# Alternatively, for simple csv file formats with only 1 header row we can add all directory contents to a single dataframe
# Combine data into dataframe
def df_from_many(dir_path, file_type=".CSV", date_col='DATE'):
    #Setup target pandas dataframe
    df = pd.DataFrame()

    #Loop over cwd/data zip files
    print(os.getcwd())
    for root, dirs, files in os.walk(dir_path):
        #print(root,dirs,files)
        for file in files:
            if file.endswith(file_type):
                print(file)
                tempdf = pd.read_csv(dir_path+file, skiprows=1)
                tempdf.index = pd.to_datetime(tempdf.loc[date_col,:])
                df = pd.concat([df, tempdf], axis=0)